## Iteration 1: initial model includes only one type of patient arrival and setting up the model logic Patient class, scenario class(expirement class), and AcuteStrokeUnit class

In [18]:
# Imports
import simpy

import numpy as np
import pandas as pd

import math

simpy.__version__

'4.1.1'

### Below is test script only

In [19]:
def print_patient_details(acute, esd):
    """
    Helper function.
    Formats length of stay in data frame
    """
    df = pd.DataFrame(np.vstack([sample_acute, sample_esd])).T
    df.columns = ["acute", "esd"]
    return df.round(2)

In [20]:
# first retest - 5 patients included.
N_PATIENTS = 5

# create two random state objects
# both distributions are using the same single stream of pseudo random numbers
rs_acute = np.random.default_rng(seed=42)
rs_esd = np.random.default_rng(seed=101)

sample_acute = rs_acute.exponential(scale=3, size=N_PATIENTS)
sample_esd = rs_esd.exponential(scale=7, size=N_PATIENTS)

print_patient_details(sample_acute, sample_esd)

,acute,esd
0,7.21,28.63
1,7.01,4.05
2,7.15,6.69
3,0.84,7.23
4,0.26,4.48


In [21]:
# second retest - 2 patients included.
N_PATIENTS = 2

# create two random state objects
# both distributions are using the same single stream of pseudo random numbers
rs_acute = np.random.default_rng(seed=42)
rs_esd = np.random.default_rng(seed=101)

sample_acute = rs_acute.exponential(scale=3, size=N_PATIENTS)
sample_esd = rs_esd.exponential(scale=7, size=N_PATIENTS)

print_patient_details(sample_acute, sample_esd)

,acute,esd
0,7.21,28.63
1,7.01,4.05


### 1. Patient generator for inter-arrival times - Using Exponential Distribution with an arrival ever 1.2 days

In [22]:
def patient_arrival_generator(env, random_seed=None):
    """
    Patient arrive every 1.2 days.

    Parameters:
    ------
    env: simpy.Environment

    random_state: int, optional (default=None)
    if set then used as random seed to control sampling.
    """
    acute_arrivals = np.random.default_rng(random_seed)

    patient_id = 1
    while True:
        inter_arrival_time = acute_arrivals.exponential(1.2)
        yield env.timeout(inter_arrival_time)

        print(f"Patient arrives at: {env.now}")
        patient_id += 1

In [23]:
# model parameters
RUN_LENGTH = 10
SEED = 42

env = simpy.Environment()
env.process(patient_arrival_generator(env, random_seed=SEED))
env.run(until=RUN_LENGTH)

print(f"End of run. Simulation clock time = {env.now}")

Patient arrives at: 2.8850503247591934
Patient arrives at: 5.688477911748537
Patient arrives at: 8.550191111597643
Patient arrives at: 8.885944259419542
Patient arrives at: 8.989669139057595
End of run. Simulation clock time = 10


---

# Start of iteration 1

### Exponential Distribution

In [24]:
class Exponential:
    """
    Convenience class for the exponential distribution.
    packages up distribution parameters, seed and random generator.
    """

    def __init__(self, mean, random_seed=None):
        """
        Constructor

        Params:
        ------
        mean: float
            The mean of the exponential distribution

        random_seed: int, optional (default=None)
            A random seed to reproduce samples.  If set to none then a unique
            sample is created.
        """
        self.rand = np.random.default_rng(seed=random_seed)
        self.mean = mean

    def sample(self, size=None):
        """
        Generate a sample from the exponential distribution

        Params:
        -------
        size: int, optional (default=None)
            the number of samples to return.  If size=None then a single
            sample is returned.
        """
        return self.rand.exponential(self.mean, size=size)

### Lognormal Distribution

In [25]:
class Lognormal:
    """
    Encapsulates a lognormal distirbution
    """

    def __init__(self, mean, stdev, random_seed=None):
        """
        Params:
        -------
        mean = mean of the lognormal distribution
        stdev = standard dev of the lognormal distribution
        """
        self.rand = np.random.default_rng(seed=random_seed)
        mu, sigma = self.normal_moments_from_lognormal(mean, stdev**2)
        self.mu = mu
        self.sigma = sigma

    def normal_moments_from_lognormal(self, m, v):
        """
        Returns mu and sigma of normal distribution
        underlying a lognormal with mean m and variance v
        source: https://blogs.sas.com/content/iml/2014/06/04/simulate-lognormal
        -data-with-specified-mean-and-variance.html

        Params:
        -------
        m = mean of lognormal distribution
        v = variance of lognormal distribution

        Returns:
        -------
        (float, float)
        """
        phi = math.sqrt(v + m**2)
        mu = math.log(m**2 / phi)
        sigma = math.sqrt(math.log(phi**2 / m**2))
        return mu, sigma

    def sample(self):
        """
        Sample from the normal distribution
        """
        return self.rand.lognormal(self.mu, self.sigma)

### 2. Class Patient

In [26]:
class Patient:
    """
    Encapsulates the process of a patient arriving at an acute stroke unit,
    waiting for a bed, staying and then leaving.

    Params:
    ------
    identifier: int
        Unique identifier for the patient.

    env: simpy.Environment
        The simulation environment.

    acute_unit: simpy.Resource
        The acute stroke unit (beds as a resource).

    los_dist: object
        Distribution object for sampling the length of stay.
    """

    def __init__(self, identifier, env, num_beds, los_dist):
        self.identifier = identifier
        self.env = env
        self.num_beds = num_beds
        self.los_dist = los_dist

    def treatment(self):
        """
        Simulates the patient treatment process:
        1. Request a bed.
        2. Stay in the unit for a sampled duration.
        3. Leave the unit.
        """
        start_wait = self.env.now  # Record when patient starts waiting

        with self.num_beds.request() as req:
            yield req  # Wait for a bed to become available

            # Calculate waiting time
            self.waiting_time = self.env.now - start_wait
            print(
                f"Patient {self.identifier}\
                 gets a bed at {self.env.now:.2f} days "
                f"(Wait time: {self.waiting_time:.2f} days)"
            )

            # Sample length of stay
            length_of_stay = self.los_dist.sample()
            yield self.env.timeout(length_of_stay)  # Simulate hospital stay

            print(
                f"Patient {self.identifier} leaves at {self.env.now:.2f} days"
            )

### 3. Class AcuteStrokeUnit & Patient Generator for Arrivals

In [27]:
class AcuteStrokeUnit:
    """
    Params:
    ------
    env: simpy.Environment
        The simulation environment.

    num_beds: int
        Number of beds in the unit.

    arrival_dist: object
        Distribution object for inter-arrival times.

    los_dist: object
        Distribution object for length of stay.
    """

    def __init__(self, env, num_beds, arrival_dist, los_dist):
        self.env = env
        self.num_beds = num_beds
        self.arrival_dist = arrival_dist
        self.los_dist = los_dist
        self.patient_count = 0

    def patient_arrival_generator(self):
        """
        Generates patients who arrive at the acute stroke unit.
        This uses an exponential distribution.
        """
        while True:
            inter_arrival_time = self.arrival_dist.sample()
            yield self.env.timeout(inter_arrival_time)  # Wait for next arrival

            self.patient_count += 1  # Increment patient count manually
            print(
                f"Patient {self.patient_count}\
                    arrives at {self.env.now:.2f} days"
            )

            # Create a new patient and start treatment
            new_patient = Patient(
                self.patient_count, self.env, self.num_beds, self.los_dist
            )

            self.env.process(new_patient.treatment())

### 4. Setting Parameters & Running Model

In [28]:
# Model parameters
RUN_LENGTH = 10  # days
N_BEDS = 10  # number of beds in the stroke unit
ARRIVAL_RATE = 1.2  # number of patients per day
MEAN_IAT = 1 / ARRIVAL_RATE  # inter-arrival time for patients (in days)
MEAN_LOS = 7  # average length of stay for a patient (in days)
STD_LOS = 2  # standard deviation for length of stay (in days)
LOS_SEED = 42  # random seed for reproducibility

ARR_SEED = 42  # random seed for reproducibility

# Create distribution objects
arrival_dist = Exponential(MEAN_IAT, random_seed=SEED)
los_dist = Lognormal(MEAN_LOS, STD_LOS, random_seed=SEED)

# Create SimPy environment
env = simpy.Environment()
bed_resoure = simpy.Resource(env, capacity=N_BEDS)

# Create Acute Stroke Unit
acute_unit = AcuteStrokeUnit(
    env, bed_resoure, arrival_dist=arrival_dist, los_dist=los_dist
)

# Start patient arrivals
env.process(acute_unit.patient_arrival_generator())

# Run the simulation
env.run(until=RUN_LENGTH)

print(f"End of run. Simulation clock time = {env.now:.2f} days")

Patient 1                    arrives at 2.00 days
Patient 1                 gets a bed at 2.00 days (Wait time: 0.00 days)
Patient 2                    arrives at 3.95 days
Patient 2                 gets a bed at 3.95 days (Wait time: 0.00 days)
Patient 3                    arrives at 5.94 days
Patient 3                 gets a bed at 5.94 days (Wait time: 0.00 days)
Patient 4                    arrives at 6.17 days
Patient 4                 gets a bed at 6.17 days (Wait time: 0.00 days)
Patient 5                    arrives at 6.24 days
Patient 5                 gets a bed at 6.24 days (Wait time: 0.00 days)
Patient 6                    arrives at 7.45 days
Patient 6                 gets a bed at 7.45 days (Wait time: 0.00 days)
Patient 7                    arrives at 8.63 days
Patient 7                 gets a bed at 8.63 days (Wait time: 0.00 days)
Patient 2 leaves at 8.98 days
Patient 1 leaves at 9.33 days
End of run. Simulation clock time = 10.00 days


---

### 1. Simplifying the large amount of inputs by using a Parameter/Scenario Class

In [29]:
class Scenario:
    """
    Stores model parameters and initialises probability distributions.

    Parameters:
    - env: SimPy environment
    - num_beds: Number of beds available in the stroke unit
    - mean_iat: Mean inter-arrival time for patients
    - mean_los: Mean length of stay for patients
    - std_los: Standard deviation of LOS
    - arrival_seed: Random seed for patient arrivals
    - los_seed: Random seed for length of stay distribution
    """

    def __init__(
        self,
        env,
        num_beds=N_BEDS,  # Number of beds
        mean_iat=MEAN_IAT,  # Mean inter-arrival time
        mean_los=MEAN_LOS,  # Mean length of stay
        std_los=STD_LOS,  # Standard deviation of LOS
        arrival_seed=ARR_SEED,  # Random seed for arrivals
        los_seed=LOS_SEED,  # Random seed for LOS distribution
    ):

        # Simulation environment
        self.env = env

        # Beds are modeled as a limited resource
        self.num_beds = num_beds = simpy.Resource(env, capacity=num_beds)

        # Distributions for arrival times and length of stay
        self.arrival_dist = Exponential(mean_iat, random_seed=arrival_seed)
        self.los_dist = Lognormal(mean_los, std_los, random_seed=los_seed)

#### 2.) Switching on Trace

In [30]:
def trace(msg):
    """
    Enables event tracing for debugging.

    Params:
    -------
    msg: str
        string to print to screen.
    """
    if TRACE:
        print(msg)

#### 3.) Patient Class

In [31]:
class Patient:  # noqa: F811
    """
    Encapsulates the process of a patient arriving at an acute stroke unit,
    waiting for a bed, staying and then leaving.

    Parameters:
    - identifier: Unique patient ID
    - env: SimPy environment
    - args: Scenario object containing hospital parameters
    """

    def __init__(self, identifier, env, args):
        self.identifier = identifier
        self.env = env
        self.num_beds = args.num_beds
        self.los_dist = args.los_dist

    def treatment(self):
        """
        Simulates the patient treatment process:
        1. Request a bed.
        2. Stay in the unit for a sampled duration.
        3. Leave the unit.
        """
        start_wait = self.env.now  # Record when patient starts waiting

        # Wait for an available bed
        with self.num_beds.request() as req:
            yield req  # Wait for a bed to become available

            # Calculate waiting time
            self.waiting_time = self.env.now - start_wait
            print(
                f"Patient {self.identifier}"
                f" gets a bed at {self.env.now:.2f} days "
                f"( Wait time: {self.waiting_time:.2f} days)"
            )

            # Sample length of stay and simulate hospital stay
            length_of_stay = self.los_dist.sample()
            yield self.env.timeout(length_of_stay)

            print(
                f"Patient {self.identifier} leaves at {self.env.now:.2f} days"
            )

#### 4.) Acute Stroke Unit Class

In [32]:
class AcuteStrokeUnit:
    """
    Parameters:
    - env: SimPy environment
    - args: Scenario object containing hospital parameters
    """

    def __init__(self, env, args):
        self.env = env
        self.patient_count = 0
        self.args = args

    def patient_arrival_generator(self):
        """
        Generates patients who arrive at the acute stroke unit.
        This uses an exponential distribution.
        """
        while True:
            inter_arrival_time = self.args.arrival_dist.sample()
            yield self.env.timeout(inter_arrival_time)  # Wait for next arrival

            self.patient_count += 1  # Increment patient count manually
            print(
                f"Patient {self.patient_count}"
                f" arrives at {self.env.now:.2f} days"
            )

            # Create a new patient and start treatment
            # new_patient = Patient(self.patient_count, self.env,
            # self.num_beds,
            # self.los_dist)
            new_patient = Patient(self.patient_count, self.env, self.args)

            self.env.process(new_patient.treatment())

#### 5.) Model Parameters

In [33]:
# Model parameters
RUN_LENGTH = 10  # simulation run length in days
N_BEDS = 1  # number of beds in the stroke unit
ARRIVAL_RATE = 1.2  # number of patients per day
MEAN_IAT = 1 / ARRIVAL_RATE  # inter-arrival time for patients (in days)
MEAN_LOS = 7  # average length of stay for a patient (in days)
STD_LOS = 2  # standard deviation for length of stay (in days)
SEED = 42  # random seed for reproducibility
TRACE = True  # toggle event tracing (on/off)


# Create SimPy environment
env = simpy.Environment()

# Create default scenario with parameters
default_args = Scenario(
    env,
    num_beds=N_BEDS,
    mean_iat=MEAN_IAT,
    mean_los=MEAN_LOS,
    std_los=STD_LOS,
    arrival_seed=ARR_SEED,
    los_seed=LOS_SEED,
)

# Create Acute Stroke Unit
acute_unit = AcuteStrokeUnit(env, default_args)

# Start patient arrivals
env.process(acute_unit.patient_arrival_generator())

# Run the simulation
env.run(until=RUN_LENGTH)

print(f"End of run. Simulation clock time = {env.now:.2f} days")

Patient 1 arrives at 2.00 days
Patient 1 gets a bed at 2.00 days ( Wait time: 0.00 days)
Patient 2 arrives at 3.95 days
Patient 3 arrives at 5.94 days
Patient 4 arrives at 6.17 days
Patient 5 arrives at 6.24 days
Patient 6 arrives at 7.45 days
Patient 7 arrives at 8.63 days
Patient 1 leaves at 9.33 days
Patient 2 gets a bed at 9.33 days ( Wait time: 5.38 days)
End of run. Simulation clock time = 10.00 days


![Iteration 1](../Images/iteration_1.png)
